<a href="https://colab.research.google.com/github/iPy849/Evolutive-Algorithms-Framework/blob/main/bioslam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimización Bioinspirada

En este proyecto, se exploran algoritmos bioinspirados para resolver problemas de optimización. Se implementan tres enfoques diferentes: Algoritmos Genéticos (AG), Optimización por Colonias de Hormigas (ACO) y Algoritmos de Enjambre de Partículas (PSO).

## Algoritmos Genéticos (AG)

Los Algoritmos Genéticos se basan en el concepto de evolución biológica y utilizan operadores como selección, cruza y mutación para buscar soluciones óptimas en un espacio de búsqueda. En este proyecto, se implementan operadores como elitismo, cruza de un punto y mutación de inversión de un bit. Estos operadores se aplican a una población de entidades binarias, donde cada entidad representa una solución potencial al problema de optimización. El objetivo es encontrar la mejor solución a través de la selección de las entidades más aptas y la combinación de sus características a través de la cruza y mutación.

## Optimización por Colonias de Hormigas (ACO)

**Nota**: Debido a limitaciones de tiempo, no se implementó el algoritmo de Optimización por Colonias de Hormigas (ACO) en este proyecto. Sin embargo, se proporciona una descripción general de los operadores y su funcionamiento para su referencia.

La Optimización por Colonias de Hormigas se inspira en el comportamiento de las colonias de hormigas en la búsqueda de caminos óptimos hacia una fuente de alimento. En este enfoque, se utiliza una matriz de feromonas para guiar la construcción de soluciones y actualizar las feromonas basándose en la calidad de las soluciones encontradas. El algoritmo ACO se basa en la idea de que las hormigas depositan feromonas en el camino que siguen, lo que permite a otras hormigas tomar decisiones informadas sobre la elección del camino.

## Algoritmos de Enjambre de Partículas (PSO)

Los Algoritmos de Enjambre de Partículas se basan en el comportamiento de los enjambres de partículas en la naturaleza, donde cada partícula representa una posible solución al problema. Las partículas exploran el espacio de búsqueda y ajustan su posición y velocidad en función de su propia mejor posición alcanzada (pbest) y la mejor posición global encontrada (gbest) por todo el enjambre. El objetivo es encontrar la mejor solución al permitir que las partículas se muevan hacia las regiones prometedoras del espacio de búsqueda.

## Conclusiones

Aunque no se implementó el algoritmo de Optimización por Colonias de Hormigas (ACO) en este proyecto, se presentaron los operadores clave y su funcionamiento para comprender el enfoque. Los Algoritmos Genéticos (AG) y los Algoritmos de Enjambre de Partículas (PSO) se implementaron con éxito, y se proporcionaron explicaciones detalladas de los operadores utilizados en cada enfoque.

Los algoritmos bioinspirados ofrecen enfoques innovadores y efectivos para abordar problemas de optimización en una amplia gama de dominios. La elección del enfoque más adecuado depende de las características específicas del problema y de las restricciones y objetivos del sistema. Al combinar diferentes técnicas bioinspiradas y adaptarlas a las necesidades del problema, es posible obtener soluciones óptimas y eficientes en una variedad de contextos.

## Importando paquetería

In [ ]:
# built - in
from enum import Enum
import string
import math
import time

# third - party
import numpy.random as random

## Definición de entidad binaria genérica:
Una entidad binaria es una plantilla o contenedor que utiliza para que la representación binaria de cierta entidad respete las condiciones de la entidad
que se quiere representar, esta clase permite generar entidad siguiendo reglas más allá de establecer límites, ejemplo:
* Números pares entre 0 y 1000
* Números primos
* Números cuya representación binaria comienza con 101

Adicionalmente esta clase nos permite trabajar con sub-representaciones/sub-entidades, representaciones embebidas en otras representaciones de manera
que podemos formar representaciones más complejas y son fáciles de extraer.

La clase BinaryEntity maneja un conjunto de verificaciones que aseguran el perfecto funcionamiento y compatibilidad con otras clases de utilidades. **Es
importante mencionar que esta clase no permite la modificación** de sus atributos luego de que es instanciada, solo es posible a través de los métodos
correspondientes implementados.

In [ ]:
class BinaryEntity(object):
    """
        Una entidad representa un entero no negativo de manera binaria.
        Una BinaryEntity no puede ser mutada excepto usando los métodos correspondientes
    """
    __sub_entities: list = None  # Sub-entidades calculadas
    __properties_locked: bool = False # Bandera que indica que la mutación de propiedades está bloqueada

    def __mutation_method(method):
        """
            Decorador para métodos que necesitan mutar una variable.
            Ejecuta el método decorado y valida el estado de la entidad.
        """
        def wrapper(self, *args, **kwargs):
            self.__properties_locked = False
            result = method(self, *args, **kwargs)
            self.__validate_entity_logic()
            self.__properties_locked = True
            return result
        return wrapper

    def __clean_entity(method):
        """
            Decorador para métodos que necesitan reiniciar el estado de la
            entidad.
        """
        def wrapper(self, *args, **kwargs):
            self.__sub_entities = None
            result = method(self, *args, **kwargs)
            return result
        return wrapper


    # NOTE: No uso el decorador porque a veces los tipos del constructor hace inferencias raras
    def __init__(
            self,
            data: int,
            entity_bit_length: int,
            sub_entities_length: list = None,
            validation_func=None,
            lower_bound: int = 0,
            upper_bound: int = None,
    ):
        """
        :rtype: BinaryEntity
        :param data: Valor contenido
        :param entity_bit_length: Cantidad de bits de su representación binaria. Si no se define \
        un valor máximo, se calcula en base a este parámetro
        :param sub_entities_length: Tamaño en bits de sub-entidades
        :param validation_func: Función de validación de la entidad, (<BinaryEntity>) -> <bool>
        :param upper_bound: Máximo valor entero admisible que puede tomar la entidad. Si este no es \
        definido se calcula como el máximo valor que puede tomar la entidad dada la cantidad de bits definida.
        :param lower_bound: Mínimo valor entero admisible que puede tomar la entidad
        """
        # Asignación de variables de instancia
        self.data = data & (2**entity_bit_length)-1 # Toma solo la cantidad de bits necesarias
        self.entity_bit_length = entity_bit_length
        self.sub_entities_length = sub_entities_length
        self.validation_func = validation_func
        self.lower_bound = lower_bound

        # ¿No hay límite superior? Asigna el máximo valor que puede tener la representación
        if upper_bound is None:
            self.upper_bound = (2 ** entity_bit_length) - 1
        else:
            self.upper_bound = upper_bound

        # Comprobar estado entidad y cerrar la mutación de propiedades
        self.__validate_entity_logic()
        self.__properties_locked = True

    def __validate_entity_logic(self):
        """Valida la lógica de las propiedades para cada mutación de estas"""
        # NOTE: Cuida que los parámetros de la entidad se definan correctamente
        assert self.lower_bound <= self.data <= self.upper_bound
        assert self.entity_bit_length > 0
        if self.sub_entities_length is not None:
            assert len(self.sub_entities_length) != 0
            assert sum(self.sub_entities_length) <= self.entity_bit_length
        if self.validation_func is not None:
            assert callable(self.validation_func)
        if self.upper_bound is not None:
            assert self.upper_bound > self.lower_bound

        # Se toman solo los bits necesario del valor de la entidad
        self.data &= (2 ** self.entity_bit_length) - 1

    def create_with_same_format(self, data: int):
        """Crea una nueva entidad a partir del formato de esta instancia"""
        return BinaryEntity(
            data,
            self.entity_bit_length,
            sub_entities_length=self.sub_entities_length,
            validation_func=self.validation_func,
            lower_bound=self.lower_bound,
            upper_bound=self.upper_bound,
        )

    def validate(self):
        """
        Comprueba que la entidad esté contenida dentro de los límites
        aceptados y cumpla la función de validación si es que fue asignada
        """
        is_in_bounds = self.upper_bound >= self.data >= self.lower_bound
        is_valid = self.validation_func(self) if self.validation_func is not None else True
        assert isinstance(is_valid, bool)
        return is_in_bounds and is_valid

    @__mutation_method
    @__clean_entity
    def set_data(self, data: int):
        """Setter para data"""
        self.data = data

    @__mutation_method
    @__clean_entity
    def set_sub_entities_length(self, sub_entities_length: list):
        """Setter para definición de sub-entidades"""
        self.sub_entities_length = sub_entities_length

    @__mutation_method
    def get_sub_entities(self) -> list | None:
        """
        Obtiene una lista de sub-entidades según los tamaños
        de sub-entidad definidos en el parámetro sub_entities_length.

        Si la sumatoria de sub_entities_length < entity_bit_length
        se devuelve una entidad adicional con los bits sobrantes
        """
        # ¿Se pueden encontrar sub-entidades?
        if self.sub_entities_length is None:
            return None
        # ¿Ya se calcularon las sub-entidades?
        if self.__sub_entities is not None:
            return self.__sub_entities

        # Crear copia de tamaño de sub-entidades en alcance local
        # para aplicar lógica de bits sobrantes
        sub_entities_length = self.sub_entities_length
        total_sub_entities_length = sum(sub_entities_length)
        if self.entity_bit_length > total_sub_entities_length:
            sub_entities_length.append(self.entity_bit_length - total_sub_entities_length)

        # Calculando sub-entidades
        sub_entities = list()
        carry = 0
        for sub_entity_length in sub_entities_length:
            data = self.data
            data >>= carry
            mask = (2 ** sub_entity_length) - 1
            data &= mask
            sub_entities.append(
                BinaryEntity(data, sub_entity_length)
            )
            carry += sub_entity_length
        self.__sub_entities = sub_entities
        return self.__sub_entities


    def __to_string(self) -> str:
        """
        Rellena con 0's la representación binaria si
        el número tiene menos bits de los especificado
        """
        rep = bin(self.data)[2:]
        if len(rep) < self.entity_bit_length:
            rep = '0' * (self.entity_bit_length - len(rep)) + rep
        return rep

    def get_entity_info(self) -> str:
        """Regresa un resumen de la entidad en formato string"""
        return 'real: %d\tbin: %s\trange: [%d,%d]\tbits: %d\thas validation: %s\tis valid: %s' % (
            self.data,
            self,
            self.lower_bound,
            self.upper_bound,
            self.entity_bit_length,
            self.validation_func is not None,
            self.validate()
        )

    def __repr__(self) -> str:
        """Sobrecarga de operador de representación"""
        return self.__to_string()

    def __gt__(self, other):
        """Sobrecarga de operador de >"""
        return self.data > other.data

    def __ge__(self, other):
        """Sobrecarga de operador de >="""
        return self.data >= other.data

    def __lt__(self, other):
        """Sobrecarga de operador de <"""
        return self.data < other.data

    def __le__(self, other):
        """Sobrecarga de operador de <="""
        return self.data <= other.data

    def __eq__(self, other):
        """Sobrecarga de operador de =="""
        return self.data == other.data

    def __ne__(self, other):
        """Sobrecarga de operador de !="""
        return self.data != other.data

    def __setattr__(self, key, value):
        """Bloquea la mutación de propiedades en base __properties_locked"""
        # Si cambia __properties_locked no aplica la lógica de bloqueo
        # __properties_locked es una propiedad privada, solo es accesible dentro de la entidad
        is_locker_mutation = key == '_BinaryEntity__properties_locked'
        if not is_locker_mutation and self.__properties_locked:
            raise Exception("Entity properties are read-only")
        super.__setattr__(self, key, value)


## Definición de entidad decimal binaria genérica

BinaryDecimalEntity es una espcialización de BinaryEntity, enfocada al manejo de números decimales. El formato utilizado no restringe la representación
a menos que así se desee. La representación consta de tres partes esenciales:
1. Bit de signo (primer bit)
2. Bits de representación entera (n bits)
3. Bits de representación entera (últimos 7 bits)

Esta clase maneja un atributo adicional **int_max_bits** que define el tamaño y valores máximos de su representación entera. Sin este valor no puede
cumplirse el comportamiento completo de la clase.

In [ ]:
class BinaryDecimalEntity(BinaryEntity):
    """
    Entidad que trata a números decimales entre [-2^27; 2^27] con tres cifras decimales.
    Siempre tiene las mismas subentidades donde representan distintas partes del número
    en el siguiente orden:
    0 - Signo del decimal (1 bit inicial)
    1 - Parte entera del decimal (n bits en medio)
    2 - Parte decimal del decimal (7 bits finales)

    Esta clase está pensada para ser un wrapper alrededor de BinaryEntity, entonces cada instancia
    de esta clase debe ser tratada como una instancia de BinaryEntity
    """
    def __init__(self, data: float, int_max_bits: int, validation_func: callable = None):
        # Parte signo
        sign_part = "0"
        sign_part_bits = 1
        # Signo negativo
        if data < 0:
            data *= -1
            sign_part = "1"

        # Parte entera del valor máximo
        upper_bound_int_part = (2 ** int_max_bits) - 1
        # Cantidad de bits del límite superior
        upper_bound_int_part_bits = len(bin(upper_bound_int_part)[2:])

        # Parte entera del valor contenido
        data_int_part = int(data - (data % 1)) # Parte entera del valor asignado
        # No puede pasarse del valor máximo definido por los bits asignados
        if data_int_part >= upper_bound_int_part:
            data_int_part = upper_bound_int_part
        data_int_part_bin =  bin(data_int_part)[2:] # Representación binaria de la parte entera del valor asignado
        # Representación binaria corregida de la parte entera del valor asignado
        fixed_data_int_part_bin = ('0' * (upper_bound_int_part_bits - len(data_int_part_bin))) + data_int_part_bin

        # Parte decimal del valor contenido, solo 2 cifras significativas
        data_dec_part = int((data % 1) * 10**2)  # Parte decimal del valor asignado
        data_dec_part_bin =  bin(data_dec_part)[2:] # Representación binaria de la parte decimal del valor asignado
        data_dec_part_bits = 7
        # Representación binaria corregida de la parte decimal del valor asignado
        fixed_data_dec_part_bin = ('0' * (data_dec_part_bits - len(data_dec_part_bin))) + data_dec_part_bin

        sub_entities_length = [sign_part_bits, upper_bound_int_part_bits, data_dec_part_bits]
        encoded_value_str =  fixed_data_dec_part_bin + fixed_data_int_part_bin + sign_part
        encoded_value = int('0b' + encoded_value_str, 2)

        # Necesito esta info para convertir y diferenciar una entidad decimal de una binaria
        self.int_max_bits = upper_bound_int_part_bits
        super().__init__(
            encoded_value,
            len(encoded_value_str),
            sub_entities_length=sub_entities_length,
            validation_func=validation_func,
        )

    def create_with_same_format(self, data: float):
        """Sobrecarga del método heredado"""
        return BinaryDecimalEntity(data, self.int_max_bits, validation_func=self.validation_func)

    def to_float(self) -> float:
        """Devuelve la información contenida como una número flotante"""
        sub_entities_bin = []
        for sub_entity in self.get_sub_entities():
            sub_entities_bin.extend(['0b'+str(sub_entity)])
        int_part = int(sub_entities_bin[1], 2)
        dec_part = int(sub_entities_bin[2], 2) * 10 ** -2
        sign_part = 1 if int(sub_entities_bin[0], 2) == 0 else -1
        return (int_part + dec_part) * sign_part

    @staticmethod
    def from_binary_entity(entity: BinaryEntity):
        """Convierte una entidad binaria a una entidad flotante binaria"""

        # Es necesario este variable porque la representación binaria de un entero no siempre
        # tiene la misma cantidad de bits representados:
        # representación 8 bits de 32 = 00010000
        # representación de python de 32 = 10000
        if not 'int_max_bits' in entity.__dict__:
            raise TypeError("the entity wasn't a BinaryDecimalEntity instance")

        # Analiza el signo del número
        sign = -1 if (entity.data & 1) == 1 else 1

        # Analiza la parte entera del número
        # Compone la máscara para extraer parte entera
        int_part_bits = (2 ** entity.int_max_bits) - 1
        int_part_mask = int_part_bits << 1
        int_part = (entity.data & int_part_mask) >> 1

        # Analiza la parte decimal del número
        # ¿Cuántos bits sobraron para los decimales? Tómalos todos
        dec_part_bits = len(str(entity)) - entity.int_max_bits - 1
        # Compone la máscara para extraer parte decimal como un número entero
        dec_part_mask = (2 ** dec_part_bits - 1) << entity.int_max_bits + 1
        dec_part = (entity.data & dec_part_mask) >> entity.int_max_bits + 1

        # Compone el número
        value = sign * (int_part + dec_part * (10 ** -2))

        # Se devuelve la representación de la entidad
        return BinaryDecimalEntity(value, entity.int_max_bits, validation_func=entity.validation_func)


### Prueba y detalles de entidad binaria

Se prueban todos los comportamientos que pueden tener las entidades bases del framework.

In [ ]:
# Esta función se pasa a la entidad como un validador de entidad de ese tipo
def entity_validation(entity: BinaryEntity) -> bool:
    pos_attacker, pos_defender = entity.get_sub_entities()
    max_pos_val = 0b101
    return pos_attacker.data <= max_pos_val and pos_defender.data <= max_pos_val

# Se crea una entidad
test_entity = BinaryEntity(
    0b100000, # Puede ser binario o decimal
    6, # Cantidad de bits de la entidad
    # Cantidad de bits de cada sub-entidad, cada sub entidad se cuenta en binario (derecha a izquierda)
    sub_entities_length=[3, 3],
    validation_func=entity_validation, # Validación adicional
    lower_bound=0, # Límite inferior de la entidad
    # upper_bound=32 # Límite superior de la entidad
)

def run_entity_test(name: str, entity: BinaryEntity):
    """Nos permite probar una entidad y su funcionamiento"""
    # Representación
    print(f'{name}: {entity}')
    # Muestra resumen de la entidad
    print(f'{name} -> info: {entity.get_entity_info()}')
    # Sub entidades que contiene una entidad
    print(f'{name} -> sub-entities: {entity.get_sub_entities()}\n')

# Se prueba una entidad
run_entity_test("Entidad de prueba", test_entity)

# Creamos un entidad nueva usando la misma configuración de la anterior
# En este caso le dimos más bits para demostrar que solo trabaja con los especificados
test_entity_2 = test_entity.create_with_same_format(0b1111100001)
run_entity_test("Entidad de prueba 2", test_entity_2)

# Una entidad no puede modificar el valor de sus propiedades a no ser que exista un método para ello
test_entity_3 = test_entity.create_with_same_format(test_entity.data)
run_entity_test("Entidad de prueba 3", test_entity_3)
try:
    test_entity_3.data = 3
except Exception:
    print("No se pudo cambiar el valor de ", test_entity_3)

test_entity_3.set_data(3)
test_entity_3.set_sub_entities_length([4,2])
run_entity_test("Entidad de prueba 3 mod", test_entity_3)


Entidad de prueba: 100000
Entidad de prueba -> info: real: 32	bin: 100000	range: [0,63]	bits: 6	has validation: True	is valid: True
Entidad de prueba -> sub-entities: [000, 100]

Entidad de prueba 2: 100001
Entidad de prueba 2 -> info: real: 33	bin: 100001	range: [0,63]	bits: 6	has validation: True	is valid: True
Entidad de prueba 2 -> sub-entities: [001, 100]

Entidad de prueba 3: 100000
Entidad de prueba 3 -> info: real: 32	bin: 100000	range: [0,63]	bits: 6	has validation: True	is valid: True
Entidad de prueba 3 -> sub-entities: [000, 100]

No se pudo cambiar el valor de  100000
Entidad de prueba 3 mod: 000011
Entidad de prueba 3 mod -> info: real: 3	bin: 000011	range: [0,63]	bits: 6	has validation: True	is valid: True
Entidad de prueba 3 mod -> sub-entities: [0011, 00]



In [ ]:
# Manipulando relaciones entre BinaryDecimalEntity y BinaryEntity
def parse_binaryDecimalEntity_to_BinaryEntity(e: BinaryDecimalEntity) -> BinaryEntity:
    """
    Cambio de tipo implícito ya que no existe typecasting en python, son
    magic methods que es otro concepto
    """
    return e

# Instancia de un BinaryDecimalEntity
binary_decimal_entity = BinaryDecimalEntity(27.68, 3)
run_entity_test("Instancia de BinaryDecimalEntity", binary_decimal_entity)
# Instancia de un BinaryDecimalEntity a partir de un BinaryDecimalEntity
binary_decimal_entity2 = binary_decimal_entity.create_with_same_format(5.0)
run_entity_test("Instancia clonada de BinaryDecimalEntity", binary_decimal_entity2)
# Instancia de un BinaryDecimalEntity a partir de un BinaryEntity
# Parseamos a BinaryEntity
binary_entity_parsed = parse_binaryDecimalEntity_to_BinaryEntity(binary_decimal_entity2)
run_entity_test("Instancia de BinaryEntity a partir de BinaryDecimalEntity:", binary_entity_parsed)
# ¿Qué pasa si intentamos parsear una entidad que nunca fue un Decimal?
try:
    binary_entity_instance = BinaryEntity(
        binary_decimal_entity.data,
        binary_decimal_entity.entity_bit_length,
        sub_entities_length=binary_decimal_entity.get_sub_entities,
        validation_func=binary_decimal_entity.validation_func,
        upper_bound=binary_decimal_entity.upper_bound,
        lower_bound=binary_decimal_entity.lower_bound
    )
    binary_entity_instance = BinaryDecimalEntity.from_binary_entity(binary_entity_instance)
    run_entity_test("Instancia de BinaryEntity", binary_entity_instance)
except TypeError:
    print("No se puede parsear directamente un BinaryEntity a BinaryDecimal Entity, falta información")




Instancia de BinaryDecimalEntity: 10000111110
Instancia de BinaryDecimalEntity -> info: real: 1086	bin: 10000111110	range: [0,2047]	bits: 11	has validation: False	is valid: True
Instancia de BinaryDecimalEntity -> sub-entities: [0, 111, 1000011]

Instancia clonada de BinaryDecimalEntity: 00000001010
Instancia clonada de BinaryDecimalEntity -> info: real: 10	bin: 00000001010	range: [0,2047]	bits: 11	has validation: False	is valid: True
Instancia clonada de BinaryDecimalEntity -> sub-entities: [0, 101, 0000000]

Instancia de BinaryEntity a partir de BinaryDecimalEntity:: 00000001010
Instancia de BinaryEntity a partir de BinaryDecimalEntity: -> info: real: 10	bin: 00000001010	range: [0,2047]	bits: 11	has validation: False	is valid: True
Instancia de BinaryEntity a partir de BinaryDecimalEntity: -> sub-entities: [0, 101, 0000000]

No se puede parsear directamente un BinaryEntity a BinaryDecimal Entity, falta información


## Generación de poblaciones
Función de generación de entidades aleatorias que se basa en una entidad plantilla para generar una nueva lista de entidades que cumplan las mismas
características y sean válidas

In [ ]:
def generate_entities(template_entity: BinaryEntity, qty: int, verbose=False) -> list[BinaryEntity|BinaryDecimalEntity]:
    """
    Genera una qty cantidad de entidades qty con las mismas características
    que la template_entity, teniendo en cuenta que cada entidad generada es
    válida.
    """
    print(f'About to create {qty} entities:')
    population = list()

    while len(population) != qty:
        # Genera información dentro de los límites
        data = 0
        # Diferencio las clases para juzgar los límites
        if isinstance(template_entity, BinaryDecimalEntity):
            data = random.randint(-2 ** template_entity.int_max_bits, 2 ** template_entity.int_max_bits) + random.random()
        else:
            data = random.randint(template_entity.lower_bound, template_entity.upper_bound )

        entity = template_entity.create_with_same_format(data)
        # Comprueba que la entidad sea válida y la muestra
        if entity.validate():
            if verbose:
                print(f'{len(population)} - {entity.get_entity_info()}')
            population.append(entity)
    print()
    return population


### Prueba y detalles generación de entidades

In [ ]:
# Las entidades no generación no tienen que ser necesariamente válidas

# Generación de entidades a partir de una entidad válida
binary_entities = generate_entities(test_entity, 5, verbose=True)

# Generación de entidades a partir de una entidad inválida
binary_decimal_entities = generate_entities(binary_decimal_entity, 5, verbose=True)


About to create 5 entities:
0 - real: 2	bin: 000010	range: [0,63]	bits: 6	has validation: True	is valid: True
1 - real: 11	bin: 001011	range: [0,63]	bits: 6	has validation: True	is valid: True
2 - real: 10	bin: 001010	range: [0,63]	bits: 6	has validation: True	is valid: True
3 - real: 11	bin: 001011	range: [0,63]	bits: 6	has validation: True	is valid: True
4 - real: 20	bin: 010100	range: [0,63]	bits: 6	has validation: True	is valid: True

About to create 5 entities:
0 - real: 705	bin: 01011000001	range: [0,2047]	bits: 11	has validation: False	is valid: True
1 - real: 1075	bin: 10000110011	range: [0,2047]	bits: 11	has validation: False	is valid: True
2 - real: 607	bin: 01001011111	range: [0,2047]	bits: 11	has validation: False	is valid: True
3 - real: 850	bin: 01101010010	range: [0,2047]	bits: 11	has validation: False	is valid: True
4 - real: 1213	bin: 10010111101	range: [0,2047]	bits: 11	has validation: False	is valid: True



## Definición de entidad dimensional binaria genérica

Una MultiDimensionalBinaryEntity es una entidad que engloba a otras entidades, a diferencia de las sub-entidades, estas se manejan como componentes
o entidades que poseen las mismas características y restricciones pero diferente valor.

Esta clase te permite trabajar con toda la instancia de MultiDimensionalBinaryEntity como una BinaryEntity usando el método **to_binary_entity**
pero a la vez elimina las restricciones y el caracter multi-dimensional de la entidad para pasar a ser tratada como un entidad con sub-entidades

In [ ]:
class MultiDimensionalBinaryEntity(object):
    """
    Representación de una entidad
    """
    def __init__(self, entities: list[BinaryEntity|BinaryDecimalEntity]):
        self.entities = entities

    def to_binary_entity(self) -> BinaryEntity:
        """Devuelve una entidad multidmensional como una entidad binaria"""
        entity_data = 0
        entities_length = []
        for i, entity in enumerate(self.entities):
            entities_length.append(len(str(entity)))
            entity_data += entity.data << (i * entities_length[i])
        return BinaryEntity(entity_data, sum(entities_length), entities_length)

    def copy(self):
        """Copia la instancia, así libera su referencia"""
        return MultiDimensionalBinaryEntity(self.entities)

    @staticmethod
    def generate_based_in(template: BinaryEntity, dims: int):
        """
            Genera una nueva instancia de MultiDimensionalBinaryEntity que genera sus valores según las dimensiones
            definidas. Utiliza la función generate_entities definida anteriormente para ello.
        """
        return MultiDimensionalBinaryEntity(generate_entities(template, dims))

    def __repr__(self):
        return "<"+"|".join([str(entity) for entity in self.entities])+">"


## Probando la entidad dimensional binaria genérica

In [ ]:
# Instancia de Entidad binaria multi-dimensional
multi_dim_binary_entity = MultiDimensionalBinaryEntity(
    generate_entities(test_entity, 3)
)

# Conviertiendo varias dimensiones en una sola
print("MultiDimensionalBinaryEntity:", multi_dim_binary_entity)
run_entity_test("MultiDimensionalBinaryEntity as BinaryEntity:", multi_dim_binary_entity.to_binary_entity())
print("MultiDimensionalBinaryEntity Copy:", multi_dim_binary_entity.copy())


About to create 3 entities:

MultiDimensionalBinaryEntity: <001001|010011|100000>
MultiDimensionalBinaryEntity as BinaryEntity:: 100000010011001001
MultiDimensionalBinaryEntity as BinaryEntity: -> info: real: 132297	bin: 100000010011001001	range: [0,262143]	bits: 18	has validation: False	is valid: True
MultiDimensionalBinaryEntity as BinaryEntity: -> sub-entities: [001001, 010011, 100000]

MultiDimensionalBinaryEntity Copy: <001001|010011|100000>


## Definición de una partícula binaria
La clase BinaryParticle es una entidad multi-dimensional que nos permite mantener el estado de las partícula para el algoritmo de PSO

In [ ]:
class BinaryParticle(MultiDimensionalBinaryEntity):
    """
        Representa una entidad de partícula binaria.
        Esta clase tiene la particularidad que sus componentes son de tipo BinaryDecimalEntity
    """
    def __init__(self, entities: list[BinaryDecimalEntity]):
        super().__init__(entities)
        self.velocity = [0 for _ in range(len(entities))]
        self.particle_best = self

    def copy(self):
        """Sobreescritura del método de la clase base para que genere el tipo propio"""
        copy = BinaryParticle(self.entities)
        if self != self.particle_best:
            copy.particle_best = self.particle_best.copy()
        return copy

    @staticmethod
    def generate_based_in(template: BinaryDecimalEntity, dims: int):
        """
            Genera una nueva instancia de BinaryParticle que genera sus valores según las dimensiones
            definidas. Utiliza la función generate_entities definida anteriormente para ello.
        """
        return BinaryParticle(generate_entities(template, dims))

    def __repr__(self):
        return "< %s - pbest: %s>" % (
                "|".join([str(entity) for entity in self.entities]),
                "|".join([str(entity) for entity in self.particle_best.entities]),
        )


In [ ]:
# Instancia de Entidad binaria multi-dimensional
binary_particle = BinaryParticle(
    generate_entities(binary_decimal_entity2, 5)
)

# Conviertiendo varias dimensiones en una sola
print("BinaryParticle:", binary_particle)
run_entity_test("BinaryParticle as BinaryEntity:", multi_dim_binary_entity.to_binary_entity())
print("BinaryParticle Copy:", binary_particle.copy())



About to create 5 entities:

BinaryParticle: < 10101011101|00100111000|10111011100|00111111101|10111101101 - pbest: 10101011101|00100111000|10111011100|00111111101|10111101101>
BinaryParticle as BinaryEntity:: 100000010011001001
BinaryParticle as BinaryEntity: -> info: real: 132297	bin: 100000010011001001	range: [0,262143]	bits: 18	has validation: False	is valid: True
BinaryParticle as BinaryEntity: -> sub-entities: [001001, 010011, 100000]

BinaryParticle Copy: < 10101011101|00100111000|10111011100|00111111101|10111101101 - pbest: 10101011101|00100111000|10111011100|00111111101|10111101101>


## Implementación de operadores
Se implementarán los diferentes operadores utilizados por los algoritmos definidos a continuación:
* Algoritmos genéticos
* Optimización colmena de partículas
* Optimización colonia de hormigas

### Implementación de las constantes/enumeradores de conjutnos de operadores
Esta clase Enum es la que define qué algoritmo se implementa en la función de correr algoritmos bio-inspirados

In [ ]:
class BioOperator(Enum):
    """Constantes enumerables de los diferentes conjuntos de operadores"""
    GA_OPERATORS = 0
    PSO_OPERATORS = 1
    ACO_OPERATORS = 2


### Implementación de operadores de algoritmos genéticos

#### Selección

##### Elitismo

In [ ]:
def selection_elitism(population: list[BinaryEntity], scores: list[float]) -> list[BinaryEntity]:
    """
    Selección de población usando elitismo, regresa la mitad de los población más apta
    :param population: población actual
    :param scores: mapeo de score de cada individuo de la población
    :return: población seleccionada
    """
    # Esto es posible gracias a que una closure empaqueta su scope con la función:
    # A mi me gusta decir que una closure secuestra el scope en que se construye
    # https://realpython.com/python-scope-legb-rule/#using-enclosing-scopes-as-closures
    return sorted(population, key=lambda x: scores[population.index(x)], reverse=True)


In [ ]:
# Probando selección elitista

# Se usa un scope separado para no generar basura
if True:
    sample_population = generate_entities(test_entity, 5) # Se genera población de 5
    sample_population_score = [random.random() for _ in sample_population] # Se inventan los scores
    selection = selection_elitism(sample_population, sample_population_score) # Selecciona usando elitismo
    print(
        "Population:\t%s\nScores:\t%s\nSelection:\t%s\n" % (
            sample_population,
            sample_population_score,
            selection,
        ),
    )

About to create 5 entities:

Population:	[010101, 011101, 001100, 010001, 010011]
Scores:	[0.2570829506384331, 0.8650714743799242, 0.8329682500547153, 0.6741488143790529, 0.8643985476029954]
Selection:	[011101, 010011, 001100, 010001, 010101]



##### Ruleta

In [ ]:
def selection_roulette(population: list[BinaryEntity], scores: list[float]) -> list[BinaryEntity]:
    """
    Selección de población usando método de ruleta, los individuos con mejores score tienen mejores
    posibilidades de ser seleccionados. Regresa la mitad de los población.
    :param population: población actual
    :param scores: mapeo de score de cada individuo de la población
    :return: población seleccionada
    """
    sum_score = sum(scores) # Total de scores

    # Calculando probabilidad de elección de individuos
    selection_probs = None
    # Si la suma de los scores es 0, todos tienen la misma posibilidad
    if sum_score == 0:
        selection_probs = [1/ len(population) for score in scores]
    else:
        selection_probs = [score/sum_score for score in scores]

    # Obtiene elementos random de la población teniendo en cuenta pesos/probabilidades
    return random.choice(population, len(population), p=selection_probs)


In [ ]:
# Probando selección ruleta

# Se usa un scope separado para no generar basura
if True:
    sample_population = generate_entities(test_entity, 5) # Se genera población de 5
    sample_population_score = [random.random() for _ in sample_population] # Se inventan los scores
    selection = selection_roulette(sample_population, sample_population_score) # Selecciona usando ruleta
    print(
        "Population:\t%s\nScores:\t%s\nSelection:\t%s\n" % (
            sample_population,
            sample_population_score,
            selection,
        ),
    )

About to create 5 entities:

Population:	[000100, 001010, 001101, 001100, 010010]
Scores:	[0.8290838406267683, 0.9010927035567851, 0.4990416207325248, 0.13030353175170173, 0.43305990012364104]
Selection:	[010010 010010 010010 001101 000100]



##### Torneo

In [ ]:
def selection_tournament(population: list[BinaryEntity], scores: list[float]) -> list[BinaryEntity]:
    """
    Selección de población usando método de torneo, los individuos compiten por la supervivencia.
    Como en los juegos del hambre.
    Regresa la mitad de los población.
    :param population: población actual
    :param scores: mapeo de score de cada individuo de la población
    :return: población seleccionada
    """
    winners = [] # Lista de individuos ganadores
    tournament_size = 3  # Cuántos compiten

    for _ in range(len(population)):
        # Seleccionar índices únicos al azar para los competidores del torneo
        competidores = random.choice(len(population), len(population) // 2, replace=False)
         # Encontrar el índice del competidor con el puntaje más bajo
        winner = max(competidores, key=lambda c: scores[c])
        winners.append(population[winner])

    return winners


In [ ]:
# Probando selección torneo

# Se usa un scope separado para no generar basura
if True:
    sample_population = generate_entities(test_entity, 5) # Se genera población de 5
    sample_population_score = [random.random() for _ in sample_population] # Se inventan los scores
    selection = selection_tournament(sample_population, sample_population_score) # Selecciona usando método torneo
    print(
        "Population:\t%s\nScores:\t%s\nSelection:\t%s\n" % (
            sample_population,
            sample_population_score,
            selection,
        ),
    )

About to create 5 entities:

Population:	[000101, 010001, 101100, 011001, 010011]
Scores:	[0.3652818570201507, 0.6107364871894106, 0.7568129966173909, 0.26656312095732815, 0.6031687178334485]
Selection:	[101100, 101100, 010001, 000101, 010011]



#### Cruza

##### Cruza de 1 punto

In [ ]:
def crossover_one_point(parent_1: BinaryEntity, parent_2: BinaryEntity) -> BinaryEntity:
    """
    Hace una cruza de 1 bit aleatorio dado dos padres, el resultado siempre se le aplica
    un módulo con el límite superior (el que nada debe, nada teme), si el resultado no es válido,
    se repite la cruza con los mismos
    padres.
    :param parent_1: padre 1
    :param parent_2: padre 2
    :return: hijo
    """
    # Bit de cruza, siempre está incluído dentro de la parte del primer padre
    cross_point = random.randint(1, parent_1.entity_bit_length-1)
    clean_left_positions = parent_1.entity_bit_length - cross_point
    # Bits del padre 1
    parent_1_part = (parent_1.data << clean_left_positions) >> clean_left_positions
    # Bits del padre 2
    parent_2_part = (parent_2.data >> cross_point)  << cross_point
    child_data = (parent_1_part | parent_2_part) % parent_1.upper_bound
    child = parent_1.create_with_same_format(child_data)

    return child


In [ ]:
# Probando cruza de 1 punto

# Se usa un scope separado para no generar basura
if True:
    test_entity_2.set_data(0b001101) # Se cambia el valor de entidad de prueba 2 para mejor visualización
    print(
        "Parent-1: %s\nParent-2: %s\nChild (1 point crossover): %s\n" % (
            test_entity,
            test_entity_2,
            crossover_one_point(test_entity, test_entity_2)
        )
    )

Parent-1: 100000
Parent-2: 001101
Child (1 point crossover): 101100



##### Cruza uniforme

In [ ]:
def crossover_uniform(parent_1: BinaryEntity, parent_2: BinaryEntity) -> BinaryEntity:
    """
    Hace una cruza uniforme entre los padres donde se cruzan aleatoriamente todos sus bits,
    el resultado siempre se le aplica un módulo con el límite superior (el que nada debe, nada teme),
    si el resultado no es válido, se repite la cruza con los mismos padres.
    :param parent_1: padre 1
    :param parent_2: padre 2
    :return: hijo
    """
    child_data = 0 # Se construye el hijo bit a bit
    for i in range(parent_1.entity_bit_length):
        parent = parent_1 if random.random() < .5 else parent_2 # ¿Qué padre uso?
        child_data += parent.data & 2 ** i # Agregar bit correspondiente del padre

    child_data %= parent_1.upper_bound
    child = parent_1.create_with_same_format(child_data)

    return child


In [ ]:
# Probando cruza uniforme

# Se usa un scope separado para no generar basura
if True:
    print(
        "Parent-1: %s\nParent-2: %s\nChild (uniform crossover): %s\n" % (
            test_entity,
            test_entity_2,
            crossover_uniform(test_entity, test_entity_2)
        )
    )

Parent-1: 100000
Parent-2: 001101
Child (uniform crossover): 000100



#### Mutación

##### Mutación de 1 bit aleatorio

In [ ]:
def mutation_one_bit_inversion(genome: BinaryEntity):
    """
    Invierte un bit aleatorio del genoma, el resultado siempre se le aplica un módulo con el
    límite superior (el que nada debe, nada teme), si el resultado no es válido, se repite la
    cruza con los mismos padres.
    :param genome: genoma a mutar
    """
    # Se crea una copia del genoma para no afectar la instancia original
    genome_copy = genome.create_with_same_format(genome.data)

    # Se recupera un bit aleatorio
    selected_bit = random.randint(0, genome_copy.entity_bit_length)
    mask = 2 ** selected_bit
    inspected_bit = (genome_copy.data & mask) >> selected_bit
    # El bit sumado se suma o sustrae de la información de la copia de genoma
    mutated_genome_data = genome_copy.data + mask * (-1 if inspected_bit == 1 else 1)
    # Se introduce la información del genoma y se comprueba que sea un genoma válido
    genome_copy.set_data(mutated_genome_data % genome_copy.upper_bound)

    genome.set_data(genome_copy.data)


In [ ]:
# Probando mutación de inversión de 1 bit

# Se usa un scope separado para no generar basura
if True:
    print("Before Mutation:\t", test_entity)
    mutation_one_bit_inversion(test_entity)
    print("After Mutation:\t", test_entity)


Before Mutation:	 100000
After Mutation:	 110000


##### Intercambio de 2 bit aleatorios

In [ ]:
def mutation_two_bit_swap(genome: BinaryEntity):
    """
    Intercambia dos bits aleatorios del genoma, el resultado siempre se le aplica un módulo con el
    límite superior (el que nada debe, nada teme), si el resultado no es válido, se repite la
    cruza con los mismos padres.
    :param genome: genoma a mutar
    """
    # Se crea una copia del genoma para no afectar la instancia original
    genome_copy = genome.create_with_same_format(genome.data)

    # Se escongen dos bits distintos
    first_bit = random.randint(0, genome_copy.entity_bit_length)
    second_bit = first_bit
    while first_bit == second_bit:
        second_bit = random.randint(0, genome_copy.entity_bit_length)

    # Se recupera cada bit a intercambiar
    first_bit_mask = 2 ** first_bit
    second_bit_mask = 2 ** second_bit
    first_bit_value = (genome_copy.data & first_bit_mask) >> first_bit
    second_bit_value = (genome_copy.data & second_bit_mask) >> second_bit

    # Si los bits a intercambiar son iguales, ignora el intercambio
    if first_bit_value != second_bit_value:
        # El bit sumado se suma o sustrae de la información de la copia de genoma
        mutated_genome_data = genome_copy.data
        mutated_genome_data += first_bit_mask * (-1 if first_bit_value == 1 else 1)
        mutated_genome_data += second_bit_mask * (-1 if second_bit_value == 1 else 1)

        # Se introduce la información del genoma y se comprueba que sea un genoma válido
        genome_copy.set_data(mutated_genome_data % genome_copy.upper_bound)

    genome.set_data(genome_copy.data)


In [ ]:
# Probando mutación de intercambio de dos bits

# Se usa un scope separado para no generar basura
if True:
    print("Before Mutation:\t", test_entity)
    mutation_two_bit_swap(test_entity)
    print("After Mutation:\t", test_entity)


Before Mutation:	 110000
After Mutation:	 011000


### Implementación de operadores de colmena de partículas

#### Calculo de velocidad
Calcula el vector de velocidad de cada dimensión para una partícula, el resultado está contenido dentro de la misma partícula la cual al ser una
instancia de clase se maneja como una refencia.

In [ ]:
def calculate_particle_velocity(particle: BinaryParticle, global_best: BinaryParticle, inertia: int, cognitive_weight: int, social_weight: int):
    """Modifica la referencia de la partícula para asignarle una nueva velocidad"""
    for dim, dim_velocity in enumerate(particle.velocity):
        particle.velocity[dim] = (
            inertia * dim_velocity +
            cognitive_weight * random.random() * (particle.particle_best.entities[dim].to_float() - particle.entities[dim].to_float()) +
            social_weight * random.random() * (global_best.entities[dim].to_float() - particle.entities[dim].to_float())
        )


#### Nueva posición

In [ ]:
def calculate_particle_position(particle: BinaryParticle):
    """Modifica la referencia de la partícula para asignarle una nueva posición"""
    for dim, dim_velocity in enumerate(particle.velocity):

        # Calcular la nueva posición
        new_position = particle.entities[dim].to_float() + particle.velocity[dim]
        real_binary_limits = (2 ** particle.entities[dim].int_max_bits) - 1

        # ¿Es menos de lo que la representación admite?
        if new_position < -real_binary_limits:
            new_position = -real_binary_limits

        # ¿Es más de lo que la representación admite?
        if new_position > real_binary_limits:
            new_position = real_binary_limits

        # Cambiar la entidad decimal en la partícula
        particle.entities[dim] = particle.entities[dim].create_with_same_format(new_position)


### Implementación de operadores de colonia de hormigas

### Implementación de conjuntos de operadores
Se definen los distintos algoritmos que se utilizarán para la solución de problemas. Cada algoritmo corresponde a uno de los valores de Enum BioOperator
## Problema:

#### Implementación de GA

##### Definición de parámetros para realización del algoritmo genético

In [ ]:
# Configuraciones de GA

# Define variables para operadores de GA
# NOTE: Los operadores siempre deben ser una tupla
SELECTION_OPERATORS = selection_elitism, selection_tournament, selection_roulette
CROSSOVER_OPERATORS = crossover_one_point, crossover_uniform
MUTATION_OPERATORS = mutation_one_bit_inversion, mutation_two_bit_swap
# Porcentaje de selección
SELECTION_RATE=.10
# Taza de mutación
MUTATION_RATE = .4


##### Implementación de algoritmo genético

In [ ]:
def ga_operators(population: list[BinaryEntity], score_func: callable) -> list[BinaryEntity]:
    """
    Ejecuta operadores aleatorios de algoritmos genéticos
    :param population: población para correr operadore
    :param score_func: función de aptitud o función objetivo. \
    La función debe tener la siguiente firma: (<BinaryEntity>) -> <float>
    :return: soluciones factible
    """
    # Referencia a globales
    global SELECTION_OPERATORS, CROSSOVER_OPERATORS, MUTATION_OPERATORS, SELECTION_RATE, MUTATION_RATE

    # Se calcula score de cada individuo
    scores = [score_func(individual) for individual in population]

    # Se ejecutan operadores de selección aleatorios, se ordenan y escogen los mejores
    # para la nueva población
    selection_operator = SELECTION_OPERATORS[random.randint(0, len(SELECTION_OPERATORS))]
    selected_population = list(selection_operator(population, scores))

    selected_scores = [score_func(individual) for individual in selected_population]
    selected_scores = sorted(population, key=lambda x: selected_scores[population.index(x)], reverse=True)
    selected_individuals_index = round(MUTATION_RATE * len(population))

    new_population = []
    new_population.extend(
        selected_population[:selected_individuals_index]
    )
    del selected_population

    # Se cruzan los individuos aleatoriamente hasta restaurar la población
    # Si el número de individuos es impar se omite una cruza aleatoria
    parent_index = 0
    while len(new_population) != len(population):
        # Se hace la cruza entre los padres
        # Selección aleatoria de operador de cruce
        crossover_operator = CROSSOVER_OPERATORS[random.randint(0, len(CROSSOVER_OPERATORS))]

        child = crossover_operator(
            population[parent_index % len(population)],
            population[(parent_index + 1) % len(population)],
        )

        # Se calcula la mutación del individuo
        if random.random() < MUTATION_RATE:
            # Selección aleatoria de operador de mutación
            mutation_operator = MUTATION_OPERATORS[random.randint(0, len(MUTATION_OPERATORS))]
            mutation_operator(child)

        # Necesitamos generar soluciones válidas
        if not child.validate():
            continue

        # Se añade el nuevo individuo
        new_population.append(child)
        parent_index += 2

    return new_population



#### Implementación de PSO

##### Definición de parámetros para realización del algoritmo de optimización de enjambre de partículas

In [ ]:
# Configuraciones de PSO

# Define variables para operadores de GA
INERTIA = 0.5
COGNITIVE_WEIGHT = 1.0
SOCIAL_WEIGHT = 1.0


##### Implementación de optimización de enjambre de partículas

In [ ]:
def pso_operators(swarm: list[BinaryParticle], global_best: BinaryParticle):
    """
    Ejecuta operadores de algoritmos de enjambre de partículas
    :param swarm: colmena de partículas
    :param global_best: mejor partícula de la colmena
    :return: colmena actualizada
    """
    global INERTIA, COGNITIVE_WEIGHT, SOCIAL_WEIGHT
    for particle in swarm:
        calculate_particle_velocity(particle, global_best, INERTIA,COGNITIVE_WEIGHT,SOCIAL_WEIGHT)
        calculate_particle_position(particle)


## Implementación del algoritmo general
Este es la función que con ciertas configuraciones aplica un algoritmo u otro, dada un serie determinada de características

In [ ]:
def bio_runner(
        genome_template: BinaryEntity,
        entity_qty: int,
        operators: BioOperator,
        problem_func: callable,
        cycles: int = 10,
        dims: int=1
) -> list[BinaryEntity, BinaryEntity, MultiDimensionalBinaryEntity, BinaryParticle]:
    # Información adicional que pueda necesitar un algoritmo
    alg_data = dict()

    # Generar <entity_qty> entidades
    entities = None
    if operators == BioOperator.GA_OPERATORS:
        entities = generate_entities(genome_template, entity_qty)
    if operators == BioOperator.PSO_OPERATORS:
        entities = [BinaryParticle.generate_based_in(genome_template, dims) for _ in range(entity_qty)]
        for entity in entities:
            if "global_best" not in alg_data.keys():
                alg_data["global_best"] = entity
            elif problem_func(alg_data["global_best"]) < problem_func(entity):
                alg_data["global_best"] = entity

    # Aplica el algoritmo
    for cycle in range(cycles):
        # Correr operadors de AG
        if operators == BioOperator.GA_OPERATORS:
            entities = ga_operators(entities, problem_func)
        # Correr operadores de PSO
        elif operators == BioOperator.PSO_OPERATORS:
            pso_operators(entities, alg_data["global_best"])
            # Actualizar particle_best y global_best
            for entity in entities:
                if problem_func(entity.particle_best) > problem_func(entity):
                    entity.particle_best = entity
                if problem_func(alg_data["global_best"]) < problem_func(entity):
                    alg_data["global_best"] = entity
        else:
            raise Exception(f'Can\'t find the specified operator: {operators}')
    # explicar resultados
    print(f'Finished {cycle} cycles')

    # organizar resultados
    entities = sorted(entities, key=lambda x: problem_func(x), reverse=True)
    return entities


# Problema 1: Problema propuesta de manera individual
# GA

Durante un partido de tenis ocurre la jugada final para determinar el punto de partido (juego, set y partido) y el jugador que ataca tiene la oportunidad de ganar ese punto lanzando la bola a una posición en la que el jugador defensor no pueda alcanzar o que se le dificulte responder ¿Cuáles son las mejores opciones para que el jugador atacante lance la bola y gane el punto?

## Espacio de búsqueda:
* La cancha de tenis está representada por dos mitades
* Cada mitad de la cancha está dividida en 6 posiciones enumeradas donde 0 ≤ x ≤ 5
* Las posiciones de cada mitad cancha están dispuestas de manera que:
 * Las posiciones pares son consideradas como posiciones de fondo de la cancha
 * Las posiciones impares son consideradas como posiciones de cercanas a la red
 * Siempre existen tres posiciones impares y tres pares
 * La númeración de las posiciones siempre tendrán los mayores valores hacia la mano hábil del jugador
* Cada mitad de cancha puede tener un disposión de la numeración diferente, siempre que cumplan con las reglas anteriores.

### Ejemplos:
**Leyenda:**
* [0-5] - posiciones
* \# - red separadora de las mitades de cancha

**Cancha donde los jugadores tienen la misma mano hábil**

||||
|:---|:----:|---:|
|0|2|4|
|1|3|5|
|#|#|#|
|5|3|1|
|4|2|0|

**Cancha donde los jugadores tienen distinta mano hábil**

||||
|:---|:----:|---:|
|0|2|4|
|1|3|5|
|#|#|#|
|1|3|5|
|0|2|4|


## Restricciones:
* Cada representación de la cancha (sus dos mitades) consideran la existencia de dos **jugadores (ja y jd)** y solo puede ser respondida con una representación de la posición de una **bola (b)**.
* En cada mitad de cancha solo puede existir un jugador.
* Cada jugador **solo puede ocupar una posición de su mitad de cancha** y según su propia distribución de posiciones en la cancha.
* Si el atacante está en la posición 3, siempre lanzará la bola a posiciones impares en la otra mitad de cancha.

## Representación de las entidades del problema
### Entidades caso
Posición p (3 bits), su valor **p está contenido en [000, 101]**, valores fuera del rango serán asignados aleatoriamente
:	Se refiere a la posición de la bola o jugadores (atacante y defensor). En el caso de los jugadores (asumiendo que tienen la misma mano hábil) las posiciones son en forma de espejo. La mano hábil del jugador determina la colocación de las posiciones, dando las posiciones más altas hacia la mano hábil.
* 001 - posición 1
* 000 - posición 0
* 101 - posición 5

Jugador j (3 bits) -> Posición
: Se refiere a una abstracción de un jugador, representando su mano hábil y la posición en que se encuentra según la distribución de posiciones asignadas
* 001 - jugador en posición 1
* 011 - jugador en posición 3

Caso jugada c (8 bits) -> Jugador atacante + Jugador defensor
: Se refiere a una abstracción de dos jugadores, donde ambos están representados por la abstracción Jugador anteriormente definida
* 001011 - atacante en posición 1 y defensor en posición 3
* 011001 - atacante en posición 3 y defensor en posición 1

### Entidades solución

Posición de la Bola b (3 bits) -> Posición
: Se refiere a la posición de la bola según la distribución de las posiciones correspondiente al jugador defensor
* 001 - posición 1
* 000 - posición 0
* 101 - posición 5

## Ejemplos de caso y solución:

**Leyenda:**
* [0-5] - posiciones
* \# - red separadora de las mitades de cancha
* ja - jugador atacante
* jd - jugador defensor

**Ejemplo 1: Cancha donde los jugadores tienen la misma mano hábil**
Caso: 001011

||||
|:---|:----:|---:|
|0|2|4|
|1 **ja**|3|5|
|#|#|#|
|5|3 **jd**|1|
|4|2|0|

**Solución:** posición 000

**Ejemplo 2: Cancha donde los jugadores tienen distinta mano hábil**
Caso: 100011

||||
|:---|:----:|---:|
|0|2|4 **ja**|
|1|3|5|
|#|#|#|
|1|3 **jd**|5|
|0|2|4|

**Solución:** posición 000

## Implementación de la solución

In [ ]:
# Ajustando variables de algoritmo genético

SELECTION_OPERATORS = selection_tournament, selection_roulette
CROSSOVER_OPERATORS = crossover_one_point,
MUTATION_OPERATORS = mutation_one_bit_inversion,

# Taza de mutación
MUTATION_RATE = .3

In [ ]:
# Información común que comparten los genomas de problema y solución

# Enum para decodificación de las posiciones
class TennisPositions(Enum):
    Fondo_Mano_No_Habil = 0
    Red_Mano_No_Habil = 1
    Fondo_Centro_Cancha = 2
    Red_Centro_Cancha = 3
    Fondo_Mano_Habil = 4
    Red_Mano_Habil=5


In [ ]:
# Se define el caso/(posición de los jugadores) como una entidad a generar

# Función de validación de un caso
def player_pos_case_validation(genome: BinaryEntity) -> bool:
    for codon in genome.get_sub_entities():
        if 0 < codon.data > 5:
            return False
    return True

# Función decodificador para caso del problema
def decode_tennis_case(genome: BinaryEntity) -> str:
    return "Caso %s:\nPos Atk: %s\nPos Def: %s" % (
        genome,
        TennisPositions(genome.get_sub_entities()[0].data).name.replace('_', ' ' ),
        TennisPositions(genome.get_sub_entities()[1].data).name.replace('_', ' ' ),
    )

# Genoma de un caso, representa la posición del jugador atacante y defensor
player_pos_case_template = BinaryEntity(
    0,
    6,
    sub_entities_length=[3,3],
    validation_func=player_pos_case_validation,
    upper_bound=0b101101,
)

player_pos_case = generate_entities(player_pos_case_template, 1, verbose=True)[0]

About to create 1 entities:
0 - real: 32	bin: 100000	range: [0,45]	bits: 6	has validation: True	is valid: True



In [ ]:
# Función de validación del genoma de la solución/bola
def winner_shot_solution_validation(genome: BinaryEntity) -> bool:
    return 0 <= genome.data <= 5

# Función de aptitud de la solución/bola (se aplica para calcular el score de los individuos)
def tennis_fit_function(genome: BinaryEntity) -> float:
    """Probabilidad de que el defensor falle la bola"""
    global player_pos_case
    # Obtener posiciones de atacante y defensor
    attacker_pos = player_pos_case.data >> 3
    defender_pos = player_pos_case.data & 0b111
    # Calcular la esquina de la cancha con mayor distancia
    max_distance = max(
        abs(defender_pos - 0),
        abs(defender_pos - 5)
    )
    # Si el atacante está en el centro de la net, siempre hará una dejada a posiciones delanteras
    ball_data = genome.data
    if attacker_pos  == 3 and genome.data % 2 == 0:
        ball_data += 1

    # Precisión del tiro
    shot_accuracy = abs(ball_data - defender_pos) / max_distance

    # Factor de disparar a mano hábil del jugador defensor
    hand_factor = .5
    if ball_data > 3: # Mayor afinidad para el jugador defensor
        hand_factor = 0
    elif ball_data < 2: # Menor afinidad para el jugador defensor
        hand_factor = 1

    # Probabilidad de que el defensor falle la bola
    return (shot_accuracy + hand_factor) / 2


# Función de decodificación de resultados
def decode_tennis_result(genome: BinaryEntity) -> str:
    return "Tiro a %s:%s - %2f" % (
        genome,
        TennisPositions(genome.data).name.replace('_', ' ' ),
        tennis_fit_function(genome)
    )


# Construyendo características del genoma de la bola/solución
ball_entity_template = BinaryEntity(
    0, # Valor dummy
    3, # Cuántos bits tiene cada solución
    validation_func=winner_shot_solution_validation,
    upper_bound=0b101 # Se define el máximo valor del genoma
)


In [ ]:
# Correr y mostrar resultados
print(BioOperator.GA_OPERATORS == BioOperator.GA_OPERATORS)

p1_exec_time = time.time().real
results = bio_runner(ball_entity_template, 5, BioOperator.GA_OPERATORS, tennis_fit_function, cycles=10)
p1_exec_time = time.time().real - p1_exec_time

print(decode_tennis_case(player_pos_case),'\nResultados:')
for result in results:
    print(decode_tennis_result(result))

True
About to create 5 entities:

Finished 9 cycles
Caso 100000:
Pos Atk: Fondo Mano No Habil
Pos Def: Fondo Mano Habil 
Resultados:
Tiro a 001:Red Mano No Habil - 0.600000
Tiro a 011:Red Centro Cancha - 0.550000
Tiro a 011:Red Centro Cancha - 0.550000
Tiro a 011:Red Centro Cancha - 0.550000
Tiro a 010:Fondo Centro Cancha - 0.450000


# Problema 2: Emparejar frases
GA

## Problema:

Dada una cadena objetivo, el objetivo es producir una cadena objetivo a partir de una cadena aleatoria de la misma longitud.

## Características:

* Los caracteres a-z y símbolos espacio se consideran genes, representados por el ínidice correspondiente a cada valor codificado en binario.

* Una cadena generada por estos caracteres se considera como cromosoma/solución/individuo.

* El valor de aptitud es el número de caracteres que coinciden de los caracteres de la cadena de destino en un índice particular.

* Por lo tanto, se da más preferencia a las personas que tienen un valor de aptitud más alto.

In [ ]:
# Ajustando variables de algoritmo genético

SELECTION_OPERATORS = selection_elitism,
CROSSOVER_OPERATORS = crossover_uniform,
MUTATION_OPERATORS = mutation_one_bit_inversion,

# Taza de mutación
MUTATION_RATE = .3

In [ ]:
# Calculando variables para definición del problema

target_string = "mundo lindo" # Palabra objetivo
search_space= string.ascii_lowercase + " " # Espacio de búsqueda
last_character_index = len(search_space) - 1 # Último índice del espacio de búsqueda
last_character_index_bin = bin(last_character_index)[2:] # Último índice del espaci`o de búsqueda en binario
max_value_bin = len(target_string) * last_character_index_bin # Valor máximo que puede tomar el genoma en binario
max_value = int(max_value_bin, 2) # Valor máximo que puede tomar el genoma


In [ ]:
# Función de validación del genoma
def phrase_match_solution_validation(genome: BinaryEntity) -> bool:
    for codon in genome.get_sub_entities():
        if codon.data > last_character_index:
            return False
    return True

# Función objetivo (se aplica para calcular el score de los individuos)
def target_function(genome: BinaryEntity) -> float:
    total_matches = len(target_string)
    matches = 0
    pairs = zip(genome.get_sub_entities(), target_string)
    for pair in pairs:
        matches += 1 if pair[1] == search_space[pair[0].data] else 0
    return matches / total_matches

# Función de decodificación de resultados
def decode_result(genome: BinaryEntity) -> str:
    phrase = ""
    for codon in genome.get_sub_entities():
        phrase += search_space[codon.data]
    return phrase

# Construyendo características del genoma (espacio de búsqueda)
template_entity = BinaryEntity(
    0, # Valor dummy
    len(max_value_bin), # Cuántos bits tiene cada solución
    sub_entities_length=[len(last_character_index_bin) for _ in target_string], # Se define el tamaño de cada codón
    validation_func=phrase_match_solution_validation,
    upper_bound=max_value # Se define el máximo valor del genoma
)


In [ ]:
# Correr y mostrar resultados
p2_exec_time = time.time().real
results = bio_runner(template_entity, 100, BioOperator.GA_OPERATORS, target_function, cycles=100)
p2_exec_time = time.time().real - p2_exec_time
# Mostrar resultados
for result in results[:10]:
    print(decode_result(result))


# Problema 3: Optimización de una función matemática
# PSO
Optimización de la función de dos dimensiones:
$$ f(a,b) = 2a^2+b^3 $$

In [ ]:
def func(p: BinaryParticle):
    return (2*p.entities[0].to_float())**2 + p.entities[1].to_float()**3

p3_exec_time = time.time().real
binary_decimal_template = BinaryDecimalEntity(0, 6) # Soluciones entre -127.99 y 127.99
p3_exec_time = time.time().real - p3_exec_time

for solution in bio_runner(binary_decimal_template, 10, BioOperator.PSO_OPERATORS, func, cycles=100, dims=2):
    print([e.to_float() for e in solution.entities])

About to create 2 entities:

About to create 2 entities:

About to create 2 entities:

About to create 2 entities:

About to create 2 entities:

About to create 2 entities:

About to create 2 entities:

About to create 2 entities:

About to create 2 entities:

About to create 2 entities:

Finished 99 cycles
[63.0, 63.0]
[63.0, 63.0]
[63.0, 63.0]
[63.0, 63.0]
[63.0, 63.0]
[63.0, 63.0]
[63.0, 63.0]
[63.0, 63.0]
[63.0, 63.0]
[63.0, 63.0]


# Problema 4: Optimización de dos funciones matemáticas
# PSO
1. Optimización de la función de dos dimensiones:
$$ f(a,b) = 2a^2+b^3 $$
2. Optimización de la función de n dimensiones:
$$ f(x_1 \cdots x_n) = 10n + \sum_{i=1}^n (x_i^2 -10cos(2\pi x_i)) $$

In [ ]:
def func1(p: BinaryParticle):
    return (2*p.entities[0].to_float())**2 + p.entities[1].to_float()**3

def func2(p: BinaryParticle):
    r = 10 * len(p.entities)
    for entity in p.entities:
        r += (entity.to_float() ** 2) - (10 * math.cos(2*math.pi*entity.to_float()))
    return r

def func(p: BinaryParticle):
    return func1(p) + func2(p)

p4_exec_time = time.time().real
binary_decimal_template = BinaryDecimalEntity(0, 6) # Soluciones entre -127.99 y 127.99
p4_exec_time = time.time().real - p4_exec_time

for solution in bio_runner(binary_decimal_template, 10, BioOperator.PSO_OPERATORS, func, cycles=100, dims=5):
    print([e.to_float() for e in solution.entities])

About to create 5 entities:

About to create 5 entities:

About to create 5 entities:

About to create 5 entities:

About to create 5 entities:

About to create 5 entities:

About to create 5 entities:

About to create 5 entities:

About to create 5 entities:

About to create 5 entities:

Finished 99 cycles
[63.0, 63.0, -63.0, 63.0, -63.0]
[63.0, 63.0, -63.0, 63.0, -63.0]
[63.0, 63.0, -63.0, 63.0, -63.0]
[63.0, 63.0, -63.0, 63.0, -63.0]
[63.0, 63.0, -63.0, 63.0, -63.0]
[63.0, 63.0, -63.0, 63.0, -63.0]
[63.0, 63.0, -63.0, 63.0, -63.0]
[63.0, 63.0, -63.0, 63.0, -63.0]
[63.0, 63.0, -63.0, 63.0, -63.0]
[63.0, 63.0, -63.0, 63.0, -63.0]


# Extra (ACO):
Como no pudo ser implementado ACO, se hizo un ejemplo de este resolviendo el problema del viajero (TSP - Traveling Salesman Problem), ya que es un problema clásico en la informática y la investigación de operaciones. Propone que, dadas las ciudades y las distancias entre cada par de ellas, el problema es encontrar la ruta más corta posible que visite cada ciudad exactamente una vez y regrese a la ciudad de origen.

In [ ]:
import random
import math

# Lista de ciudades como coordenadas (x, y)
ciudades = [(1, 1), (8, 1), (4, 6), (7, 10), (9, 7), (2, 4), (3, 9)]

def calcular_distancia(ciudad1, ciudad2):
    return math.sqrt((ciudad1[0] - ciudad2[0]) ** 2 + (ciudad1[1] - ciudad2[1]) ** 2)

def crear_matriz_distancias(ciudades):
    matriz_distancias = []
    for i in range(len(ciudades)):
        fila = []
        for j in range(len(ciudades)):
            fila.append(calcular_distancia(ciudades[i], ciudades[j]))
        matriz_distancias.append(fila)
    return matriz_distancias

matriz_distancias = crear_matriz_distancias(ciudades)

def aco_tsp(numero_hormigas, numero_iteraciones, coef_evaporacion):
    matriz_feromonas = [[1 for _ in range(len(ciudades))] for _ in range(len(ciudades))]
    mejor_camino = None
    mejor_distancia = float("inf")

    for _ in range(numero_iteraciones):
        caminos = []
        for _ in range(numero_hormigas):
            ciudades_visitadas = [random.randint(0, len(ciudades) - 1)]
            while len(ciudades_visitadas) < len(ciudades):
                probabilidades = []
                for ciudad in range(len(ciudades)):
                    if ciudad not in ciudades_visitadas:
                        feromona = matriz_feromonas[ciudades_visitadas[-1]][ciudad]
                        distancia = matriz_distancias[ciudades_visitadas[-1]][ciudad]
                        probabilidad = feromona / distancia
                        probabilidades.append((probabilidad, ciudad))
                probabilidades.sort(reverse=True)
                ciudades_visitadas.append(probabilidades[0][1])
            caminos.append(ciudades_visitadas)

        # Evaporación de feromonas
        for i in range(len(matriz_feromonas)):
            for j in range(len(matriz_feromonas[i])):
                matriz_feromonas[i][j] *= (1 - coef_evaporacion)

        for camino in caminos:
            distancia = sum([matriz_distancias[camino[i]][camino[i + 1]] for i in range(len(camino) - 1)] + [matriz_distancias[camino[-1]][camino[0]]])
            if distancia < mejor_distancia:
                mejor_distancia = distancia
                mejor_camino = camino

            # Actualizar feromonas
            for i in range(len(camino) - 1):
                matriz_feromonas[camino[i]][camino[i + 1]] += 1 / distancia
            matriz_feromonas[camino[-1]][camino[0]] += 1 / distancia

    return mejor_camino, mejor_distancia

aco_exec_time = time.time().real
mejor_camino, mejor_distancia = aco_tsp(10, 100, 0.1)
aco_exec_time = time.time().real - aco_exec_time
print("Mejor camino: ", mejor_camino)
print("Mejor distancia: ", mejor_distancia)


Mejor camino:  [4, 3, 6, 2, 5, 0, 1]
Mejor distancia:  29.964401876462816


En este código, además de la matriz de distancias, se introduce una matriz de feromonas. Cada hormiga actualiza la matriz de feromonas al completar su recorrido. La cantidad de feromonas depositadas en cada tramo es inversamente proporcional a la longitud del recorrido. También se introduce la evaporación de feromonas, que disminuye la cantidad de feromonas en cada tramo después de cada iteración. Además, la elección de la siguiente ciudad por cada hormiga ahora depende tanto de la distancia hasta la ciudad como de la cantidad de feromonas en el tramo hasta la ciudad.

# Se muestran los tiempos de ejecución de cada algoritmo:

In [ ]:
execution_time = (
    "Tiempo de ejecución del problema 1 GA - %.5fs" % p1_exec_time,
    "Tiempo de ejecución del problema 2 GA - %.5fs" % p2_exec_time,
    "Tiempo de ejecución del problema 3 PSO - %.5fs" % p3_exec_time,
    "Tiempo de ejecución del problema 4 PSO - %.5fs" % p4_exec_time,
    "Tiempo de ejecución del problema TSP - ACO - %.5fs" % aco_exec_time,
)
for i in execution_time:
    print(i)

Tiempo de ejecución del problema 1 GA - 0.00561s
Tiempo de ejecución del problema 2 GA - 4.15349s
Tiempo de ejecución del problema 3 PSO - 0.00015s
Tiempo de ejecución del problema 4 PSO - 0.00017s
Tiempo de ejecución del problema TSP - ACO - 0.03279s
